In [2]:
!pip install --upgrade plotly  # use plotly >= 5

Defaulting to user installation because normal site-packages is not writeable


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import *

import pandas as pd
import numpy as np
import plotly.express as px

# Generation of waste by waste category, hazardousness and NACE Rev. 2 activity 

[Data Explorere](https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=env_wasgen&lang=en)

[Abfallwirtschaft in der EU: Zahlen und Fakten](https://www.europarl.europa.eu/news/de/headlines/society/20180328STO00751/abfallwirtschaft-in-der-eu-zahlen-und-fakten)

[Circular economy](https://www.europarl.europa.eu/thinktank/infographics/circulareconomy/public/index.html)

[Waste and recycling](https://ec.europa.eu/environment/topics/waste-and-recycling_en)

![](https://www.europarl.europa.eu/resources/library/images/20201201PHT92816/20201201PHT92816_original.jpg)



## Read data

In [6]:
file = "data/env_wasgen_old.xls"

In [3]:
file = "data/env_wasgen_new.xlsx"

Benötigte Codes:

- A: Land- und Forstwirtschaft, Fischerei
- EP_HH: Households
- C: Herstellung von Waren
- B: Bergbau und Gewinnung von Steinen und Erden
- F: Baugewerbe
- {G477, G-U-X_G4677, D, E} = Category\\{TOTAL_HH}: Andere 

In [12]:
codes = get_header_codes_from_excel(file)

In [15]:
print_codes(file)

/Users/miay/miniconda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Category:  freq
---------
A: Annual

Category:  unit
---------
KG_HAB: Kilograms per capita
T: Tonne

Category:  hazard
---------
HAZ_NHAZ: Hazardous and non-hazardous - Total
HAZ: Hazardous
NHAZ: Non-hazardous

Category:  nace_r2
---------
A: Agriculture, forestry and fishing
B: Mining and quarrying
C: Manufacturing
D: Electricity, gas, steam and air conditioning supply
E: Water supply; sewerage, waste management and remediation activities
F: Construction
G-U_X_G4677: Services (except wholesale of waste and scrap)
G4677: Wholesale of waste and scrap
EP_HH: Households
TOTAL_HH: All NACE activities plus households

Category:  waste
---------
TOTAL: Total waste

Category:  geo
---------
EU27_2020: European Union - 27 countries (from 2020)
EU28: European Union - 28 countries (2013-2020)
BE: Belgium
BG: Bulgaria
CZ: Czechia
DK: Denmark
DE: Germany (until 1990 former territory of the FRG)
EE: Estonia
IE: Ireland
EL: Greece
ES: Spain
FR: France
HR: Croatia
IT: Italy
CY: Cyprus
LV: Latvia
LT:

In [6]:
df = pd.read_parquet("data/env_wasgen_combined.parquet")

In [7]:
df

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU27_2020,2004,146.0,KG_HAB,HAZ_NHAZ,TOTAL,A
EU28,2004,130.0,KG_HAB,HAZ_NHAZ,TOTAL,A
BE,2004,114.0,KG_HAB,HAZ_NHAZ,TOTAL,A
BG,2004,94.0,KG_HAB,HAZ_NHAZ,TOTAL,A
CZ,2004,122.0,KG_HAB,HAZ_NHAZ,TOTAL,A
...,...,...,...,...,...,...
AL,2018,NaN,T,NHAZ,TOT_X_MIN,TOTAL_HH
RS,2018,11780914.0,T,NHAZ,TOT_X_MIN,TOTAL_HH
TR,2018,74786520.0,T,NHAZ,TOT_X_MIN,TOTAL_HH


## Waste generation 2016

In [8]:
# Gesamte Müllproduktion 2016

total_waste_gen_2016 = df.query("geo == 'EU28' and year == 2016 and unit == 'T' and hazard == 'HAZ_NHAZ' and waste=='TOTAL' and nace_r2=='TOTAL_HH'")['value'][0]
f"{total_waste_gen_2016:,}"  # 2,5 Milliarden Tonnen -> korrekt

'2,536,280,000.0'

In [9]:
# Behalte nace_r2 als Kategorie 

waste_gen_2016 = df.query("geo == 'EU28' and year == 2016 and unit == 'T' and hazard == 'HAZ_NHAZ' and waste=='TOTAL'")
waste_gen_2016

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU28,2016,2.071000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2016,6.411600e+08,T,HAZ_NHAZ,TOTAL,B
EU28,2016,2.620900e+08,T,HAZ_NHAZ,TOTAL,C
EU28,2016,7.798000e+07,T,HAZ_NHAZ,TOTAL,D
EU28,2016,2.548800e+08,T,HAZ_NHAZ,TOTAL,E
EU28,2016,9.237400e+08,T,HAZ_NHAZ,TOTAL,F
EU28,2016,1.153900e+08,T,HAZ_NHAZ,TOTAL,G-U_X_G4677
EU28,2016,2.575000e+07,T,HAZ_NHAZ,TOTAL,G4677
EU28,2016,2.146900e+08,T,HAZ_NHAZ,TOTAL,EP_HH


In [10]:
# Definiere Müllproduktion "Andere" als other

other = ['G4677', 'G-U_X_G4677', 'D', 'E']
waste_other = waste_gen_2016.query("nace_r2 in @other")['value'].sum()

In [11]:
# Füge Kategorie "OTHER" hinzu

waste_gen_2016 = waste_gen_2016.append({'year':2016, 'value':waste_other, 'unit':'T', 'hazard':'HAZ_NHAZ', 'nace_r2':'OTHER', 'waste':'TOTAL'}, ignore_index = True)
waste_gen_2016

,year,value,unit,hazard,waste,nace_r2
0,2016,2.071000e+07,T,HAZ_NHAZ,TOTAL,A
1,2016,6.411600e+08,T,HAZ_NHAZ,TOTAL,B
2,2016,2.620900e+08,T,HAZ_NHAZ,TOTAL,C
3,2016,7.798000e+07,T,HAZ_NHAZ,TOTAL,D
4,2016,2.548800e+08,T,HAZ_NHAZ,TOTAL,E
5,2016,9.237400e+08,T,HAZ_NHAZ,TOTAL,F
6,2016,1.153900e+08,T,HAZ_NHAZ,TOTAL,G-U_X_G4677
7,2016,2.575000e+07,T,HAZ_NHAZ,TOTAL,G4677
8,2016,2.146900e+08,T,HAZ_NHAZ,TOTAL,EP_HH
9,2016,2.536280e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH


In [12]:
# Berechne prozentualen Anteil jeweiliger Kategorien

waste_gen_2016['percentage'] = waste_gen_2016['value'] * 100 / total_waste_gen_2016
waste_gen_2016

,year,value,unit,hazard,waste,nace_r2,percentage
0,2016,2.071000e+07,T,HAZ_NHAZ,TOTAL,A,0.816550
1,2016,6.411600e+08,T,HAZ_NHAZ,TOTAL,B,25.279543
2,2016,2.620900e+08,T,HAZ_NHAZ,TOTAL,C,10.333638
3,2016,7.798000e+07,T,HAZ_NHAZ,TOTAL,D,3.074582
4,2016,2.548800e+08,T,HAZ_NHAZ,TOTAL,E,10.049364
5,2016,9.237400e+08,T,HAZ_NHAZ,TOTAL,F,36.421058
6,2016,1.153900e+08,T,HAZ_NHAZ,TOTAL,G-U_X_G4677,4.549577
7,2016,2.575000e+07,T,HAZ_NHAZ,TOTAL,G4677,1.015266
8,2016,2.146900e+08,T,HAZ_NHAZ,TOTAL,EP_HH,8.464759
9,2016,2.536280e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH,100.000000


## Visualization

In [13]:
codes["NACE_R2"]

{'EP_HH': 'Households',
 'B': 'Mining and quarrying',
 'F': 'Construction',
 'G4677': 'Wholesale of waste and scrap',
 'D': 'Electricity, gas, steam and air conditioning supply',
 'A': 'Agriculture, forestry and fishing',
 'C': 'Manufacturing',
 'G-U_X_G4677': 'Services (except wholesale of waste and scrap)',
 'E': 'Water supply; sewerage, waste management and remediation activities',
 'TOTAL_HH': 'All NACE activities plus households'}

In [14]:
fig = px.bar(
    waste_gen_2016.sort_values("percentage", ascending=False).replace(codes["NACE_R2"]), 
    x="nace_r2",
    y="percentage",
    title="Anteil der Wirtschaftssektoren am Abfallaufkommen in der EU für 2016",
    text="percentage",
    labels={"nace_r2": "Wirtschaftskategorien", "percentage": "Prozentualer Anteil"},
    height=800,
)
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')

fig.show()

In [15]:
fig = px.bar(
    waste_gen_2016.sort_values("percentage", ascending=False).replace(codes["NACE_R2"]), 
    x="nace_r2",
    y="percentage",
    title="Anteil der Wirtschaftssektoren am Abfallaufkommen in der EU für 2016",
    text="percentage",
    labels={"nace_r2": "Wirtschaftskategorien", "percentage": "Prozentualer Anteil"},
    height=800,
    color="nace_r2",
)
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')

fig.show()

In [16]:
fig = px.bar(
    waste_gen_2016.query("nace_r2 != 'TOTAL_HH'").sort_values("percentage", ascending=False).replace(codes["NACE_R2"]),
    x=np.repeat(1, len(waste_gen_2016) - 1),
    y="percentage",
    title="Anteil der Wirtschaftssektoren am Abfallaufkommen in der EU für 2016",
    text="percentage",
    labels={"nace_r2": "Wirtschaftskategorien", "percentage": "Prozentualer Anteil"},
    height=800,
    color="nace_r2",
)
fig.update_traces(texttemplate='%{text:.1f}')

fig.show()

In [17]:
other = ['G4677', 'G-U_X_G4677', 'D', 'E']
waste_gen_2016 = waste_gen_2016.query("nace_r2 not in @other")

In [18]:
fig = px.bar(
    waste_gen_2016.query("nace_r2 != 'TOTAL_HH'").sort_values("percentage", ascending=False).replace(codes["NACE_R2"]),
    x=np.repeat(1, len(waste_gen_2016) - 1),
    y="percentage",
    title="Anteil der Wirtschaftssektoren am Abfallaufkommen in der EU für 2016",
    text="nace_r2",
    labels={"nace_r2": "Wirtschaftskategorien", "percentage": "Prozentualer Anteil"},
    height=800,
    color="nace_r2",
)
fig.update_layout(showlegend=False)
fig.show()

# Waste generation within EU

In [19]:
total_waste_gen_eu = df.query("geo == 'EU28' and unit == 'T' and hazard == 'HAZ_NHAZ' and waste=='TOTAL' and nace_r2=='TOTAL_HH'")
total_waste_gen_eu

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU28,2004,2.547590e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2006,2.567270e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2008,2.427000e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2010,2.454720e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2012,2.483990e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2014,2.507100e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2016,2.536280e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2018,2.618970e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH


In [24]:
fig = px.bar(
    total_waste_gen_eu,
    x="year",
    y="value",
    title="Müllproduktion in der EU von 2004 bis 2018",
    labels={"year": "", "value": "Tonnen"},
    height=700,
    text="value"
)

fig.update_traces(texttemplate='%{text:,}')
fig.show()

In [25]:
## Berücksichtige nace_r2-Kategorie

total_waste_gen = df.query("geo == 'EU28' and unit == 'T' and hazard == 'HAZ_NHAZ' and waste=='TOTAL'" )
total_waste_gen

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU28,2004,6.407000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2006,5.770000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2008,4.653000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2010,2.100000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2012,2.138000e+07,T,HAZ_NHAZ,TOTAL,A
...,...,...,...,...,...,...
EU28,2010,2.454720e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2012,2.483990e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2014,2.507100e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH


In [30]:
fig = px.bar(
    total_waste_gen.query("nace_r2 != 'TOTAL_HH'").replace(codes["NACE_R2"]),
    y="year",
    x="value",
    orientation="h",
    title="Müllproduktion in der EU von 2004 bis 2018 für verschiedene Wirtschaftszweige",
    labels={"year": "", "value": "Tonnen"},
    color="nace_r2",
    # text="value",
)
# fig.update_traces(texttemplate='%{text:,}')
fig.show()

## Add sub-category

In [31]:
total_waste_gen = df.query("geo == 'EU28' and unit == 'T' and waste=='TOTAL'" )
total_waste_gen

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU28,2004,6.407000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2006,5.770000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2008,4.653000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2010,2.100000e+07,T,HAZ_NHAZ,TOTAL,A
EU28,2012,2.138000e+07,T,HAZ_NHAZ,TOTAL,A
...,...,...,...,...,...,...
EU28,2010,2.358660e+09,T,NHAZ,TOTAL,TOTAL_HH
EU28,2012,2.388620e+09,T,NHAZ,TOTAL,TOTAL_HH
EU28,2014,2.412230e+09,T,NHAZ,TOTAL,TOTAL_HH


In [38]:
# waste generation with hazard and non_hazard by nace_r2

total_waste_gen = total_waste_gen[total_waste_gen.hazard != "HAZ_NHAZ"]
fig = px.bar(
    total_waste_gen[total_waste_gen.nace_r2 != "TOTAL_HH"],
    y="year",
    x="value",
    orientation= "h",
    title="Müllproduktion in der EU von 2004 bis 2018 für verschiedene Wirtschaftszweige und Gefahrenstoffe",
    labels={"year": "", "value": "Tonnen"},
    color="nace_r2",
    pattern_shape="hazard",
)
# hover_name='test', hover_data=['HAZ']
fig.show()

In [36]:
fig = px.bar(
    total_waste_gen[total_waste_gen.nace_r2 != "TOTAL_HH"],
    x="year",
    y="value",
    orientation="v",
    title="Müllproduktion in der EU von 2004 bis 2018 für verschiedene Wirtschaftszweige und Gefahrenstoffe",
    labels={"year": "", "value": "Tonnen"},
    color="nace_r2",
    pattern_shape="hazard",
)

fig.show()

## Animation

In [43]:
fig = px.bar(
    total_waste_gen[total_waste_gen.nace_r2 != "TOTAL_HH"].replace(codes["NACE_R2"]).sort_values(by="value",ascending=False),
    x="nace_r2",
    y="value",
    barmode="relative",
    orientation= "v",
    hover_name='hazard',
    hover_data=['hazard'],
    animation_frame="year",
    range_y = [0,1000000000],
    title="Müllproduktion in der EU von 2004 bis 2018 für verschiedene Wirtschaftszweige und Gefahrenstoffe",
    labels={"year": "", "value": "Tonnen"},
    color="nace_r2",
    height=800,
)
fig.show()

In [71]:
fig = px.bar(
    total_waste_gen.replace(codes["NACE_R2"]),
    x="year",
    y="value",
    orientation= "v",
    title="Müllproduktion in der EU von 2004 bis 2018 für verschiedene Wirtschaftszweige",
    labels={"year": "", "value": "Tonnen"},
    facet_row="nace_r2",
    #facet_col="nace_r2",
    #facet_col_wrap=2,
    height=2000,
)
fig.update_yaxes(matches=None)  # ohne diese Zeile können wir alle Wirtschaftszweige gut                                         vergleichen, da die y-Achse immer dieselbe Einheit hat,                                       aber mit dieser Zeile ist es einfacher, jeden Zweig                                           individuell zu analysieren
fig.show()

# Waste generation within EU over years

In [58]:
total_waste_gen_eu

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU28,2004,2.547590e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2006,2.567270e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2008,2.427000e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2010,2.454720e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2012,2.483990e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2014,2.507100e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2016,2.536280e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2018,2.618970e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH


In [59]:
fig_1 = px.bar(total_waste_gen_eu, x="year", y="value", orientation= "v", text="value", title="Müllproduktion in der EU von 2004 bis 2018 absolut in Tonnen", labels={"year": "", "value": "Tonnen"})
fig_1.update_traces(texttemplate='%{text:,}', textposition='inside')
fig_1.show()

In [60]:
total_waste_gen_eu_kg_hab = df.query("geo == 'EU28' and unit == 'KG_HAB' and hazard == 'HAZ_NHAZ' and waste=='TOTAL' and nace_r2=='TOTAL_HH'")
total_waste_gen_eu_kg_hab

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU28,2004,5161.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2006,5162.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2008,4842.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2010,4872.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2012,4923.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2014,4936.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2016,4966.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2018,5105.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH


In [61]:
fig_2 = px.bar(total_waste_gen_eu_kg_hab, x="year", y="value", orientation= "v", text="value", title="Müllproduktion in der EU von 2004 bis 2018 relativ in KG pro Person", labels={"year": "", "value": "Kilogramm pro Person"})
fig_2.update_traces(texttemplate='%{text:,}', textposition='inside')
fig_2.show()

## Waste generation in comparison T und KG/HAB

In [63]:
total_waste_gen_eu_combined = total_waste_gen_eu.append(total_waste_gen_eu_kg_hab)
total_waste_gen_eu_combined

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU28,2004,2.547590e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2006,2.567270e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2008,2.427000e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2010,2.454720e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2012,2.483990e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2014,2.507100e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2016,2.536280e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2018,2.618970e+09,T,HAZ_NHAZ,TOTAL,TOTAL_HH
EU28,2004,5.161000e+03,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH


In [72]:
fig = px.bar(
    total_waste_gen_eu_combined.replace(codes["UNIT"]),
    x="year",
    y="value",
    facet_col="unit",
    color="unit",
    labels={"year": ""}
)
fig.update_yaxes(matches=None)
fig.show()

## Waste generation in comparison by states

In [73]:
waste_gen_by_states = df.query("unit == 'KG_HAB' and hazard == 'HAZ_NHAZ' and waste=='TOTAL'")
waste_gen_by_states

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
EU27_2020,2004,146.0,KG_HAB,HAZ_NHAZ,TOTAL,A
EU28,2004,130.0,KG_HAB,HAZ_NHAZ,TOTAL,A
BE,2004,114.0,KG_HAB,HAZ_NHAZ,TOTAL,A
BG,2004,94.0,KG_HAB,HAZ_NHAZ,TOTAL,A
CZ,2004,122.0,KG_HAB,HAZ_NHAZ,TOTAL,A
...,...,...,...,...,...,...
AL,2018,NaN,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
RS,2018,7319.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
TR,2018,1195.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH


In [74]:
# Entferne die "Nicht-Länder"

other = ['EU28', 'EU27_2020']
waste_gen_by_states = waste_gen_by_states.query("geo not in @other")
waste_gen_by_states

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
BE,2004,114.0,KG_HAB,HAZ_NHAZ,TOTAL,A
BG,2004,94.0,KG_HAB,HAZ_NHAZ,TOTAL,A
CZ,2004,122.0,KG_HAB,HAZ_NHAZ,TOTAL,A
DK,2004,4.0,KG_HAB,HAZ_NHAZ,TOTAL,A
DE,2004,15.0,KG_HAB,HAZ_NHAZ,TOTAL,A
...,...,...,...,...,...,...
AL,2018,NaN,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
RS,2018,7319.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
TR,2018,1195.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH


## Waste generation in 2004

In [77]:
#waste_gen_by_states[waste_gen_by_states.year == 2004]
waste_gen_by_states_2004 = waste_gen_by_states.query("year == 2004 and nace_r2 == 'TOTAL_HH'").sort_values("value", ascending=False)
waste_gen_by_states_2004

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
BG,2004,26050.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
LU,2004,18153.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
RO,2004,17215.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
EE,2004,15310.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
FI,2004,13333.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
SE,2004,10203.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
MT,2004,7840.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
AT,2004,6488.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
IE,2004,6019.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH


In [78]:
# Länder mit höchster Müllproduktion

first_states = waste_gen_by_states_2004.iloc[:4].index.tolist()
first_states

['BG', 'LU', 'RO', 'EE']

In [79]:
# Länder mit niedrigster Müllproduktion

last_states = waste_gen_by_states_2004[~waste_gen_by_states_2004.value.isna()].iloc[-4:].index.tolist()
last_states

['HR', 'NO', 'TR', 'LV']

In [80]:
states = first_states + last_states
states

['BG', 'LU', 'RO', 'EE', 'HR', 'NO', 'TR', 'LV']

In [81]:
waste_gen_by_states = waste_gen_by_states.query("geo in @last_states")
waste_gen_by_states

,year,value,unit,hazard,waste,nace_r2
geo,,,,,,
HR,2004,28.0,KG_HAB,HAZ_NHAZ,TOTAL,A
LV,2004,92.0,KG_HAB,HAZ_NHAZ,TOTAL,A
NO,2004,32.0,KG_HAB,HAZ_NHAZ,TOTAL,A
TR,2004,0.0,KG_HAB,HAZ_NHAZ,TOTAL,A
HR,2006,17.0,KG_HAB,HAZ_NHAZ,TOTAL,A
...,...,...,...,...,...,...
TR,2016,953.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
HR,2018,1355.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
LV,2018,920.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH


In [82]:
waste_gen_by_states = waste_gen_by_states.reset_index()
waste_gen_by_states

,geo,year,value,unit,hazard,waste,nace_r2
0,HR,2004,28.0,KG_HAB,HAZ_NHAZ,TOTAL,A
1,LV,2004,92.0,KG_HAB,HAZ_NHAZ,TOTAL,A
2,NO,2004,32.0,KG_HAB,HAZ_NHAZ,TOTAL,A
3,TR,2004,0.0,KG_HAB,HAZ_NHAZ,TOTAL,A
4,HR,2006,17.0,KG_HAB,HAZ_NHAZ,TOTAL,A
...,...,...,...,...,...,...,...
315,TR,2016,953.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
316,HR,2018,1355.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
317,LV,2018,920.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH
318,NO,2018,2662.0,KG_HAB,HAZ_NHAZ,TOTAL,TOTAL_HH


In [84]:
fig = px.bar(
    waste_gen_by_states[waste_gen_by_states.nace_r2 != "TOTAL_HH"],
    x="geo",
    y="value",
    barmode="group", # bars are placed beside each other
    orientation= "v",
    animation_frame="year",
    title="Müllproduktion über die Jahre für die 4 Länder mit dem geringsten Abfallaufkommen",
    labels={"geo": "", "value": "Kilogramm pro Person"},
    color="nace_r2"
)
fig.show()

In [102]:
fig = px.bar(
    waste_gen_by_states,
    x="year",
    y="value",
    orientation= "v",
    title="Müllproduktion im Ländervergleich über die Jahre",
    labels={"year": "year", "value": "KG_HAB"},
    facet_col="geo",
    facet_col_wrap=2
)
fig.show()